Install Packages/Libraries

In [57]:
%pip install --upgrade pip
%pip install torch torchvision torchaudio

# Transformers & datasets
%pip install pandas
%pip install datasets
%pip install peft
%pip install transformers
%pip install transformers[torch]
%pip install 'accelerate>=0.26.0'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: transformers[torch]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


Import Packages/Libraries

In [58]:
import os, math, torch, pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer, TrainingArguments, TrainerCallback
)
from peft import get_peft_model, LoraConfig, TaskType

Configurations

In [59]:
USE_LORA = True
DATA_DIR     = "outputs/bigcode-ts-output-4000-formatted"
METADATA_CSV = "outputs/bigcode-ts-output-4000-types.csv"
OUTPUT_DIR   = "outputs/typescriptmate-model"
BATCH_SIZE   = 4
MAX_LENGTH   = 512
EPOCHS       = 3
LR           = 5e-5
GRAD_CLIP    = 0.0
SEED         = 42

Count number of TypeScipt files in folder

In [60]:
file_count = sum(len(files) for _, _, files in os.walk(DATA_DIR))
print("Total files:", file_count)

Total files: 4247


Check if MPS (Accelerated PyTorch Training for Apple Silicon) is supported

In [61]:
import torch
print(torch.backends.mps.is_available())  # Should be True
print(torch.backends.mps.is_built())       # Should be True


True
True


Load metadata for Type Awareness

In [62]:
df = pd.read_csv(METADATA_CSV)
for col in ["TypeAliases", "Interfaces"]:
    if col in df.columns:
        df[col] = df[col].fillna("")
print(f"Loaded {len(df)} metadata rows")

Loaded 4453 metadata rows


Attach file text

In [63]:
dataset_meta = Dataset.from_pandas(df, preserve_index=False)

def add_text(example):
    path = example["File"]
    if not os.path.isabs(path):
        path = os.path.join(DATA_DIR, path)
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            example["text"] = f.read()
    except:
        example["text"] = ""
    return example

dataset_meta = dataset_meta.map(add_text, batched=False)

Map: 100%|██████████| 4453/4453 [00:00<00:00, 4734.67 examples/s]


Filter bad examples

In [64]:
dataset_meta = dataset_meta.filter(lambda ex: ex["text"].strip() != "")
print("Non-empty examples:", len(dataset_meta))

Filter: 100%|██████████| 4453/4453 [00:00<00:00, 94811.19 examples/s]

Non-empty examples: 4243


Split and filter train and validation data for annotated examples

In [65]:
splits = dataset_meta.train_test_split(test_size=0.1, seed=SEED)
datasets = DatasetDict({
    "train": splits["train"].filter(lambda ex: ex["TypeAliases"] or ex["Interfaces"]),
    "validation": splits["test"].filter(lambda ex: ex["TypeAliases"] or ex["Interfaces"])
})
print("Filtered split:")
print("  • train:", len(datasets["train"]))
print("  • validation:", len(datasets["validation"]))

Filter: 100%|██████████| 425/425 [00:00<00:00, 39142.29 examples/s]

Filtered split:
  • train: 550
  • validation: 76


Tokenize

In [66]:
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )

to_remove = df.columns.tolist() + ["text"]
tokenized = datasets.map(
    tokenize_fn,
    batched=True,
    remove_columns=to_remove
)

Map: 100%|██████████| 76/76 [00:00<00:00, 343.96 examples/s]


Sanity check on tokens

In [67]:
valid_count = sum(
    any(tok != tokenizer.eos_token_id for tok in ex["input_ids"])
    for ex in tokenized["train"]
)
print(f"Usable tokenized examples: {valid_count} / {len(tokenized['train'])}")

Usable tokenized examples: 550 / 550


Collator & base Model

In [68]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
base_model = AutoModelForCausalLM.from_pretrained("gpt2")

Apply LoRA if enabled

In [69]:
if USE_LORA:
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=4,
        lora_alpha=16,
        lora_dropout=0.0,
        target_modules=["c_attn"]  # you can inspect model to try others
    )
    model = get_peft_model(base_model, lora_config)
    model.print_trainable_parameters()
else:
    model = base_model

trainable params: 147,456 || all params: 124,587,264 || trainable%: 0.1184


/Users/zfir/miniforge3/envs/dissertation_mps/lib/python3.13/site-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Move model to supported device

In [70]:
device = (
    torch.device("mps") if torch.backends.mps.is_available()
    else torch.device("cuda") if torch.cuda.is_available()
    else torch.device("cpu")
)
print("Using device:", device)

model.to(torch.float32)
model.to(device)

Using device: mps


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitud

TrainingArguments

In [71]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    max_grad_norm=GRAD_CLIP,
    logging_steps=100,
    save_steps=500,
)

Trainer with loss logger

In [72]:
class LossLogger(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        print("LOGS:", logs)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    data_collator=data_collator,
    callbacks=[LossLogger()]
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Training

In [73]:
trainer.train()

/Users/zfir/miniforge3/envs/dissertation_mps/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
100,3.034900
200,2.712300
300,2.582600
400,2.545200


LOGS: {'loss': 3.0349, 'learning_rate': 3.804347826086957e-05, 'epoch': 0.7246376811594203}
LOGS: {'loss': 2.7123, 'learning_rate': 2.5966183574879227e-05, 'epoch': 1.4492753623188406}
LOGS: {'loss': 2.5826, 'learning_rate': 1.388888888888889e-05, 'epoch': 2.1739130434782608}
LOGS: {'loss': 2.5452, 'learning_rate': 1.8115942028985508e-06, 'epoch': 2.898550724637681}
LOGS: {'train_runtime': 331.3815, 'train_samples_per_second': 4.979, 'train_steps_per_second': 1.249, 'total_flos': 431879277772800.0, 'train_loss': 2.711522346533439, 'epoch': 3.0}


TrainOutput(global_step=414, training_loss=2.711522346533439, metrics={'train_runtime': 331.3815, 'train_samples_per_second': 4.979, 'train_steps_per_second': 1.249, 'total_flos': 431879277772800.0, 'train_loss': 2.711522346533439, 'epoch': 3.0})

Save model and tokenizer

In [74]:
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

('outputs/typescriptmate-model/tokenizer_config.json',
 'outputs/typescriptmate-model/special_tokens_map.json',
 'outputs/typescriptmate-model/vocab.json',
 'outputs/typescriptmate-model/merges.txt',
 'outputs/typescriptmate-model/added_tokens.json',
 'outputs/typescriptmate-model/tokenizer.json')

Basic evaluation

In [75]:
eval_results = trainer.evaluate()

print("Full eval results:", eval_results)

if eval_results.get("eval_loss") is not None and not math.isnan(eval_results["eval_loss"]):
    print("Validation Perplexity: ", math.exp(eval_results["eval_loss"]))
else:
    print("NaN eval loss")

/Users/zfir/miniforge3/envs/dissertation_mps/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


LOGS: {'eval_loss': 2.371490001678467, 'eval_runtime': 5.5538, 'eval_samples_per_second': 13.684, 'eval_steps_per_second': 3.421, 'epoch': 3.0}
Full eval results: {'eval_loss': 2.371490001678467, 'eval_runtime': 5.5538, 'eval_samples_per_second': 13.684, 'eval_steps_per_second': 3.421, 'epoch': 3.0}
Validation Perplexity:  10.713343297136609
